# Handcrafted Feature-Based Classification (COIL-20)
### Pipeline: HOG → Scaling → PCA vs LDA → SVM → Evaluation


In [ ]:
!pip install scikit-image

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix


## Set Dataset Path
If using Colab, update this path after cloning your repo.


In [ ]:
dataset_path = "dataset"  # Change if needed


## Load Images and Labels


In [ ]:
images = []
labels = []

for filename in os.listdir(dataset_path):
    if filename.endswith('.png'):
        img_path = os.path.join(dataset_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img / 255.0
        images.append(img)
        label = int(filename.split('__')[0].replace('obj',''))
        labels.append(label)

X = np.array(images)
y = np.array(labels)

print('Dataset shape:', X.shape)


## Extract HOG Features


In [ ]:
hog_features = []

for img in X:
    features = hog(img, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), block_norm='L2-Hys')
    hog_features.append(features)

X_features = np.array(hog_features)
print('HOG feature shape:', X_features.shape)


## Feature Scaling


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)


## Train-Test Split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# PCA + SVM


In [ ]:
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model_pca = SVC(kernel='linear')
model_pca.fit(X_train_pca, y_train)

y_pred_pca = model_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
print('PCA + SVM Accuracy:', accuracy_pca)


In [ ]:
cm_pca = confusion_matrix(y_test, y_pred_pca)

plt.figure()
plt.imshow(cm_pca)
plt.title('Confusion Matrix - PCA')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


# LDA + SVM


In [ ]:
lda = LDA(n_components=19)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

model_lda = SVC(kernel='linear')
model_lda.fit(X_train_lda, y_train)

y_pred_lda = model_lda.predict(X_test_lda)
accuracy_lda = accuracy_score(y_test, y_pred_lda)
print('LDA + SVM Accuracy:', accuracy_lda)


In [ ]:
cm_lda = confusion_matrix(y_test, y_pred_lda)

plt.figure()
plt.imshow(cm_lda)
plt.title('Confusion Matrix - LDA')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
